In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('data/RIA_before_2025.csv').dropna().reset_index(drop=True).reset_index()
df

,index,date,year,month,day,text
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, есл..."
1,1,2020-01-01 00:54:16,2020,1,1,Ким Чен Ын пообещал скоро представить новое ст...
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили ..."
3,3,2020-01-01 12:55:30,2020,1,1,Столкновения произошли в Гонконге во время сог...
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что нак..."
...,...,...,...,...,...,...
168730,168730,2024-12-31 18:21:42,2024,12,31,Заявка на транзит российского газа через Украи...
168731,168731,2024-12-31 19:45:52,2024,12,31,\n\n▪️Российские войска нанесли групповой удар...
168732,168732,2024-12-31 21:02:19,2024,12,31,"❗️Путин заслушал доклады начальника Генштаба, ..."
168733,168733,2024-12-31 21:05:02,2024,12,31,Повторные выбросы нефтепродуктов зафиксированы...


In [3]:
# df = df.drop(columns=['multi_labels', 'hier_label']).copy()

In [4]:
# import json
# import re
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import pandas as pd

# MODEL_NAME = "yandex/YandexGPT-5-Lite-8B-pretrain"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     device_map="cuda",
#     torch_dtype="auto",
# )

In [5]:
# def generate_categories(news_text):
#     prompt = f"""
# Ты — редактор новостного агрегатора.  Твоя задача - категоризировать новость. Предоставь JSON:

# - **multi_labels**: Список (list) до 3 основных тем (Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости).
# - **hier_label**: Список (list) [тема, подтема]. `тема` - первая из `multi_labels`, `подтема` - её уточнение (например, "Политика", "Внешняя политика").

# Входные данные - словарь (dict) с ключами: 'text' (текст новости).

# Текст новости: {news_text}

# Ответ:
# """
#     model.config.pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

#     input_ids = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
#     outputs = model.generate(**input_ids, max_length=2048)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print(response)

#     # Извлекаем JSON из текста
#     json_match = re.search(r"\{.*\}", response, re.DOTALL)
#     if json_match:
#         json_text = json_match.group(0)
#         try:
#             return json.loads(json_text)
#         except json.JSONDecodeError:
#             return {"multi_labels": [], "hier_label": []}  # Возвращаем пустую структуру при ошибке
#     else:
#         return {"multi_labels": [], "hier_label": []}


# def process_first_n_rows(df, n):
#     """Обрабатывает первые n строк DataFrame, добавляя категории."""

#     # Создаем копию первых n строк, чтобы избежать SettingWithCopyWarning
#     df_subset = df.head(n).copy()

#     # Применяем функцию generate_categories к подмножеству DataFrame
#     df_subset["categories"] = df_subset["text"].apply(generate_categories)
#     df_subset["multi_labels"] = df_subset["categories"].apply(lambda x: x["multi_labels"])
#     df_subset["hier_label"] = df_subset["categories"].apply(lambda x: x["hier_label"])
#     df_subset.drop(columns=["categories"], inplace=True)

#     return df_subset

# N = 10
# df_processed = process_first_n_rows(df, N)

# df = df.drop(index=df.index[:N])  #удаляем старые N строк, которые мы обработали
# df = pd.concat([df_processed, df]).reset_index(drop=True) #объединяем обработанные строки с остальными

# df

In [6]:
# import json
# import re
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import pandas as pd

# # --- Модифицированный код для модели T-lite ---
# MODEL_NAME = "t-tech/T-lite-it-1.0"

# # Инициализация модели и токенизатора
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_NAME,
#     device_map="auto",
#     torch_dtype="auto",
# )

# def generate_categories(news_text):
#     # Формируем системный промпт и запрос пользователя
#     messages = [
#         {
#             "role": "system",
#             "content": """Ты — редактор новостного агрегатора в России. Твоя задача: категоризировать новости для российских читателей, создав две колонки: multi_labels и hier_label.
            
#             multi_labels: Список [,] из наиболее значимых категорий новости.
#             hier_label: Список [,] - Двухуровневая иерархия для первой категории.
            
#             Необходим ответ без пояснений строго в формате JSON."""
#         },
#         {
#             "role": "user", 
#             "content": f"Текст новости: {news_text}"
#         }
#     ]
    
#     # Применяем чатовый шаблон
#     text = tokenizer.apply_chat_template(
#         messages,
#         tokenize=False,
#         add_generation_prompt=True
#     )
    
#     # Токенизация и генерация
#     model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
#     generated_ids = model.generate(
#         **model_inputs,
#         max_new_tokens=128,
#         pad_token_id=tokenizer.eos_token_id
#     )
    
#     # Извлекаем сгенерированный текст
#     generated_ids = [
#         output_ids[len(input_ids):] 
#         for input_ids, output_ids 
#         in zip(model_inputs.input_ids, generated_ids)
#     ]
#     response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     print(response)
#     # Парсинг JSON
#     try:
#         json_match = re.search(r"\{.*\}", response, re.DOTALL)
#         return json.loads(json_match.group(0)) if json_match else {"multi_labels": [], "hier_label": []}
#     except json.JSONDecodeError:
#         return {"multi_labels": [], "hier_label": []}

# # Остальной код обработки DataFrame остается без изменений
# def process_first_n_rows(df, n):
#     df_subset = df.head(n).copy()
#     df_subset["categories"] = df_subset["text"].apply(generate_categories)
#     df_subset["multi_labels"] = df_subset["categories"].apply(lambda x: x["multi_labels"])
#     df_subset["hier_label"] = df_subset["categories"].apply(lambda x: x["hier_label"])
#     df_subset.drop(columns=["categories"], inplace=True)
#     return df_subset

# # Параметры обработки
# N = 10  # Сколько строк обрабатывать за раз

# # Основной цикл обработки
# df_processed = process_first_n_rows(df, N)
# print("Обработанные данные:")
# print(df_processed)

# # Обновление основного DataFrame (как в твоем оригинальном коде)
# df = df.drop(index=df.index[:N])
# df = pd.concat([df_processed, df]).reset_index(drop=True)
# df

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
df.head(10)

,index,date,year,month,day,text
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, если кто-то из американцев в Ираке пострадает от нападений: «Иран понесет полную ответственность за гибель людей или ущерб, причиненный какому-либо из наших объектов. Они заплатят очень большую цену! Это не предупреждение, а угроза. Счастливого нового года!»\n\nРанее протестующие проникли за внешний периметр ограды посольства США в Багдаде. США обвинили в нападении власти Ирана, Тегеран это отрицает."
1,1,2020-01-01 00:54:16,2020,1,1,"Ким Чен Ын пообещал скоро представить новое стратегическое оружие КНДР и заявил, что будет продолжать разработку стратегического оружия, пока враждебная политика США не будет устранена"
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили Новый год, участвуя в праздничных мероприятиях, сообщило МВД РФ. Нарушений правопорядка не допущено. В МВД добавили, что общественный порядок охраняли около 67,4 тысячи полицейских."
3,3,2020-01-01 12:55:30,2020,1,1,"Столкновения произошли в Гонконге во время согласованной акции, полиция применила слезоточивый газ для разгона протестующих, сообщает газета South China Morning Post."
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что накануне отшлепал по руке одну не в меру активную паломницу.\n\nВо вторник вечером женщина не на шутку рассердила папу Франциска, который вышел на площадь Святого Петра в Ватикане, чтобы полюбоваться установленными там под елью рождественскими яслями и поприветствовать верующих. Понтифик по обычаю пожимал присутствующим руки и уже собирался вернуться в папские покои, когда паломница неожиданно схватила его за правую руку и бесцеремонно потянула ее к себе.\n\n"
5,5,2020-01-01 15:45:51,2020,1,1,"Более 75,5 тысяч пассажиров встретили Новый год в поездах дальнего следования, сообщила ""Федеральная пассажирская компания"""
6,6,2020-01-01 17:12:14,2020,1,1,"В России в ночь на 1 января , сообщает «Столото». Теперь счастливчика ждут для оформления выигрыша. Пока известно лишь то, что билет был куплен в Москве."
7,7,2020-01-01 19:38:39,2020,1,1,"Украинские силовики заявили, что в новом году , ранен военный.\n\nНесмотря на то что ""нормандские лидеры"" призывали до конца 2019 года обеспечить прекращение огня."
8,8,2020-01-02 00:51:53,2020,1,2,"Экс-глава Национальной баскетбольной ассоциации Дэвид Стерн скончался в среду в возрасте 77 лет.\n\nСпортивный менеджер встал у руля NBA в 1984 году и возглавлял ее три десятка лет, сделав одной из самых богатых спортивных лиг в мире - третьей в Северной Америке после NFL (американский футбол) и MLB (бейсбол). В декабре Стерн был госпитализирован из-за кровоизлияния в мозг."
9,9,2020-01-02 03:52:11,2020,1,2,"МЧС хочет из зон чрезвычайных ситуаций при соответствующем решении властей.\n\nВ законопроекте, в частности, предусмотрены ""возможность принудительной эвакуации населения из зон ЧС"" и ""введение ответственности за несвоевременное представление информации о ЧС""."


# Gemini

In [ ]:
import pandas as pd
import json
import re
import time
import math
import google.generativeai as genai

# --- НАСТРОЙКИ ---
API_KEY_FILE = "config.py" # Файл с вашим API ключом (API_KEY = "...")
MODEL_NAME = "gemini-2.5-flash-preview-04-17" #"gemini-2.0-flash"

# Целевое количество токенов для ВСЕГО запроса к API
TARGET_TOKENS_PER_BATCH = 15000
# Примерный дневной лимит запросов (если у вас есть бесплатный лимит)
DAILY_REQUEST_LIMIT = 480
# Задержка между запросами к API в секундах (для соблюдения RPM лимитов)
REQUEST_DELAY_SECONDS = 1.1
SAMPLE_SIZE = 1000

# --- КОНЕЦ НАСТРОЕК ---

# Загрузка API ключа
try:
    from importlib import import_module
    config_module = import_module(API_KEY_FILE.replace('.py', ''))
    API_KEY = config_module.API_KEY
except (ImportError, AttributeError):
    print(f"Ошибка: Не удалось загрузить API_KEY из {API_KEY_FILE}.")
    print(f"Убедитесь, что файл {API_KEY_FILE} существует и содержит переменную API_KEY.")
    exit()

# Конфигурация Google GenAI
try:
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Ошибка конфигурации Google GenAI: {e}")
    exit()

generation_config = {
  "temperature": 0.3,
  "max_output_tokens": 65536, #8192
  "response_mime_type": "application/json",
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]

gemini_model = genai.GenerativeModel(
    model_name=MODEL_NAME,
    generation_config=generation_config,
    safety_settings=safety_settings
)

PROMPT_INSTRUCTION_TEMPLATE = """
Ты — редактор новостного агрегатора. Твоя задача - категоризировать список новостей.
Для каждой новости из списка предоставь JSON-объект.
Каждый такой объект должен содержать:
- "id": оригинальный идентификатор новости (целое число), который был предоставлен во входных данных.
- "multi_labels": Список (list) до 3 основных тем новости. Допустимые темы: Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости.
- "hier_label": Список (list) из двух элементов: [тема, подтема]. `тема` - это первая тема из `multi_labels`. `подтема` - это уточнение для этой основной темы (например, если тема "Политика", подтема может быть "Внешняя политика" или "Выборы").

Входные данные - это JSON-массив объектов, где каждый объект имеет "id" и "text".
Твой ответ ДОЛЖЕН БЫТЬ JSON-массивом, где каждый элемент массива - это JSON-объект с полями "id", "multi_labels", "hier_label" для соответствующей входной новости.
Убедись, что идентификаторы 'id' в твоем ответе точно соответствуют идентификаторам 'id' из входного списка новостей.
Не включай в ответ ничего, кроме этого JSON-массива.
Новости для обработки:
{news_json_payload}
Ответ (только JSON-массив):
"""

try:
    base_prompt_for_counting = PROMPT_INSTRUCTION_TEMPLATE.format(news_json_payload="[]")
    TOKENS_FOR_BASE_PROMPT = gemini_model.count_tokens(base_prompt_for_counting).total_tokens
    print(f"Токены, занимаемые базовой инструкцией промпта: {TOKENS_FOR_BASE_PROMPT}")
except Exception as e:
    print(f"Не удалось посчитать токены для базового промпта: {e}. Используем примерное значение 300.")
    TOKENS_FOR_BASE_PROMPT = 300

def generate_categories_batch_gemini(news_batch_with_ids_and_tokens):
    news_input_for_prompt = [{"id": item["id"], "text": item["text"]} for item in news_batch_with_ids_and_tokens]
    news_input_json_string = json.dumps(news_input_for_prompt, ensure_ascii=False, indent=2)
    full_prompt = PROMPT_INSTRUCTION_TEMPLATE.format(news_json_payload=news_input_json_string)
    empty_result_for_batch = [{"id": item["id"], "multi_labels": [], "hier_label": []} for item in news_batch_with_ids_and_tokens]

    try:
        response = gemini_model.generate_content(full_prompt)
        if not response.parts:
            print(f"Предупреждение: Пустой ответ от API (возможно, заблокировано). IDs: {[item['id'] for item in news_batch_with_ids_and_tokens]}")
            if hasattr(response, 'prompt_feedback') and response.prompt_feedback:
                 print(f"  Причина: {response.prompt_feedback}")
            return empty_result_for_batch

        print(f"DEBUG: response.parts существует. Количество частей: {len(response.parts) if response.parts else 0}")
        for i, part in enumerate(response.parts):
            print(f"DEBUG: Часть {i}: {type(part)}")
            if hasattr(part, 'text'):
                print(f"  Текст части {i}: '{part.text[:200]}...' (всего {len(part.text)} символов)")

        response_text = response.text
        print(f"DEBUG: response_text после response.text: '{response_text[:200]}...' (всего {len(response_text)} символов)")
        try:
            parsed_json = json.loads(response_text)
            if not isinstance(parsed_json, list):
                print(f"Ошибка: Ответ API не является JSON-массивом. Ответ: {response_text[:200]}...")
                return empty_result_for_batch

            # Валидация и сопоставление результатов
            results_map = {item.get("id"): item for item in parsed_json if isinstance(item, dict)}
            final_results = []
            for requested_item in news_batch_with_ids_and_tokens:
                req_id = requested_item["id"]
                if req_id in results_map:
                    res_item = results_map[req_id]
                    if isinstance(res_item.get("multi_labels"), list) and isinstance(res_item.get("hier_label"), list):
                        final_results.append(res_item)
                    else:
                        print(f"Предупреждение: Некорректная структура категорий для id {req_id} в ответе API. Заполняем пустыми.")
                        final_results.append({"id": req_id, "multi_labels": [], "hier_label": []})
                else:
                    print(f"Предупреждение: ID {req_id} отсутствует в ответе API. Заполняем пустыми.")
                    final_results.append({"id": req_id, "multi_labels": [], "hier_label": []})
            return final_results

        except json.JSONDecodeError as e:
            print(f"Ошибка декодирования JSON из ответа API: {e}\nТекст ответа (начало): {response_text[:500]}")
            return empty_result_for_batch

    except Exception as e:
        print(f"Критическая ошибка при вызове API Gemini или обработке ответа: {e}")
        # Дополнительная информация об ошибке, если доступна
        if hasattr(e, 'message'): print(f"  Сообщение ошибки: {e.message}")
        if "TOKEN_LIMIT_EXCEEDED" in str(e) or "payload size exceeds" in str(e):
            print("  Рекомендация: Размер батча (количество токенов) слишком велик. Попробуйте уменьшить TARGET_TOKENS_PER_BATCH.")
        return empty_result_for_batch

def process_dataframe_in_token_batches(df_input,
                                       target_tokens_total=TARGET_TOKENS_PER_BATCH,
                                       base_prompt_tokens=TOKENS_FOR_BASE_PROMPT,
                                       daily_request_limit=DAILY_REQUEST_LIMIT,
                                       request_delay=REQUEST_DELAY_SECONDS):
    df = df_input.copy()
    all_processed_results = []
    current_batch_items = []
    current_batch_estimated_tokens = base_prompt_tokens
    requests_count = 0

    df_results = df.copy()
    df_results["multi_labels"] = pd.NA
    df_results["hier_label"] = pd.NA
    df_results["multi_labels"] = df_results["multi_labels"].astype('object')
    df_results["hier_label"] = df_results["hier_label"].astype('object')

    print("Предварительный подсчет токенов для каждой новости...")
    news_with_tokens_info = []
    JSON_OVERHEAD_PER_ITEM_APPROX = 15 # Примерный оверхед на JSON-обертку элемента
    try:
        example_json_item_tokens = gemini_model.count_tokens(json.dumps({"id": 1234567890, "text": ""}) + ",").total_tokens
        JSON_OVERHEAD_PER_ITEM_APPROX = example_json_item_tokens
    except Exception: pass # Используем значение по умолчанию, если подсчет не удался
    print(f"  Примерный JSON оверхед на элемент: {JSON_OVERHEAD_PER_ITEM_APPROX} токенов")

    start_time_counting = time.time()
    for i, (idx, row) in enumerate(df.iterrows()):
        if (i + 1) % 500 == 0:
            print(f"  Подсчет токенов: обработано {i + 1}/{len(df)}...")
        news_text = str(row["text"])
        tokens_for_item = (len(news_text) // 4) + JSON_OVERHEAD_PER_ITEM_APPROX # Грубая оценка для скорости
        try: # Попытка точного подсчета
            tokens_for_text_only = gemini_model.count_tokens(news_text).total_tokens
            tokens_for_item = tokens_for_text_only + JSON_OVERHEAD_PER_ITEM_APPROX
        except Exception: pass # Используем грубую оценку, если точный подсчет не удался
        news_with_tokens_info.append({"id": idx, "text": news_text, "tokens": tokens_for_item})
    print(f"Предварительный подсчет токенов завершен. Заняло: {time.time() - start_time_counting:.2f}s")

    for news_item in news_with_tokens_info:
        if requests_count >= daily_request_limit:
            print(f"Достигнут дневной лимит запросов ({requests_count}). Обработка остановлена.")
            break

        item_tokens = news_item["tokens"]
        if current_batch_items and (current_batch_estimated_tokens + item_tokens > target_tokens_total):
            print(f"Отправка батча ({len(current_batch_items)} новостей, ~{current_batch_estimated_tokens} токенов). Запрос #{requests_count + 1}")
            batch_results = generate_categories_batch_gemini(current_batch_items)
            requests_count += 1
            if batch_results: all_processed_results.extend(batch_results)
            current_batch_items = []
            current_batch_estimated_tokens = base_prompt_tokens
            if requests_count < daily_request_limit and request_delay > 0: time.sleep(request_delay)

        if current_batch_estimated_tokens + item_tokens <= target_tokens_total:
            current_batch_items.append(news_item)
            current_batch_estimated_tokens += item_tokens
        else:
            if item_tokens > (target_tokens_total - base_prompt_tokens):
                print(f"Предупреждение: Новость id {news_item['id']} ({item_tokens} токенов) слишком велика для батча (лимит полезной нагрузки: {target_tokens_total - base_prompt_tokens}). Пропускается.")
                continue
            if current_batch_items: # Отправить предыдущий почти полный батч
                print(f"Отправка батча ({len(current_batch_items)} новостей, ~{current_batch_estimated_tokens} токенов). Запрос #{requests_count + 1}")
                batch_results = generate_categories_batch_gemini(current_batch_items)
                requests_count += 1
                if batch_results: all_processed_results.extend(batch_results)
                if requests_count < daily_request_limit and request_delay > 0: time.sleep(request_delay)
            current_batch_items = [news_item]
            current_batch_estimated_tokens = base_prompt_tokens + item_tokens

    if current_batch_items and requests_count < daily_request_limit:
        print(f"Отправка последнего батча ({len(current_batch_items)} новостей, ~{current_batch_estimated_tokens} токенов). Запрос #{requests_count + 1}")
        batch_results = generate_categories_batch_gemini(current_batch_items)
        requests_count += 1
        if batch_results: all_processed_results.extend(batch_results)

    print(f"Всего сделано запросов к API: {requests_count}")

    if all_processed_results:
        results_df = pd.DataFrame(all_processed_results)
        if not results_df.empty and 'id' in results_df.columns:
            # Приведение типов ID для корректного мержа
            if pd.api.types.is_numeric_dtype(df_results.index.dtype):
                results_df['id'] = pd.to_numeric(results_df['id'], errors='coerce')
                results_df.dropna(subset=['id'], inplace=True) # Удаляем ID, которые не смогли стать числами
                if not results_df.empty: # Проверка после dropna
                    try:
                        results_df['id'] = results_df['id'].astype(df_results.index.dtype)
                    except ValueError as e:
                        print(f"Предупреждение: не удалось привести 'id' в results_df к типу индекса {df_results.index.dtype}. Ошибка: {e}. Попытка int.")
                        try:
                            results_df['id'] = results_df['id'].astype(int) # Общий числовой тип
                        except ValueError:
                             print("Предупреждение: не удалось привести 'id' к int. Мерж может быть некорректным.")


            elif df_results.index.dtype == 'object': # Если индекс строковый
                 results_df['id'] = results_df['id'].astype(str)
            # else: оставляем как есть или добавляем обработку других типов индекса

            for _, result_row in results_df.iterrows():
                api_id = result_row.get('id')
                # Проверка на NaN/NaT после pd.to_numeric(errors='coerce')
                if pd.isna(api_id):
                    # print(f"Предупреждение: Пропущен результат с некорректным ID (NaN/NaT): {result_row}")
                    continue

                if api_id in df_results.index:
                    df_results.at[api_id, "multi_labels"] = result_row.get("multi_labels", [])
                    df_results.at[api_id, "hier_label"] = result_row.get("hier_label", [])
                else:
                    # Этот print может быть очень многословным, если проблема массовая
                    # print(f"Предупреждение: ID {api_id} (тип: {type(api_id)}) из API не найден в df_results.index (тип: {df_results.index.dtype}).")
                    pass # Пропускаем тихо, если ID не найден, чтобы не засорять вывод
        else:
            print("Предупреждение: DataFrame с результатами пуст или не содержит колонку 'id'.")
    else:
        print("Не было получено результатов от API для мержа.")
    return df_results

df_main = df.copy()


# Возьмем сэмпл для теста или весь DataFrame, если он небольшой
if len(df_main) > SAMPLE_SIZE:
    df_to_process = df_main.sample(n=SAMPLE_SIZE, random_state=42).copy()
    print(f"Используется выборка из {SAMPLE_SIZE} строк для обработки.")
else:
    df_to_process = df_main.copy()
    print(f"Используется полный DataFrame ({len(df_main)} строк) для обработки.")

# Настройка для конкретного запуска (можно переопределить глобальные)
# Для реального запуска используйте TARGET_TOKENS_PER_BATCH
# Для теста можно поставить меньше, чтобы увидеть несколько батчей
CURRENT_RUN_TARGET_TOKENS = TARGET_TOKENS_PER_BATCH # или, например, 30000 для теста разбиения

print(f"\n--- Запуск обработки с целевым лимитом токенов на батч: {CURRENT_RUN_TARGET_TOKENS} ---")

start_processing_time = time.time()
df_processed = process_dataframe_in_token_batches(
    df_to_process,
    target_tokens_total=CURRENT_RUN_TARGET_TOKENS,
    base_prompt_tokens=TOKENS_FOR_BASE_PROMPT
)
end_processing_time = time.time()
print(f"Общее время обработки: {end_processing_time - start_processing_time:.2f} секунд")

df_processed

Токены, занимаемые базовой инструкцией промпта: 346
Используется выборка из 1000 строк для обработки.

--- Запуск обработки с целевым лимитом токенов на батч: 15000 ---
Предварительный подсчет токенов для каждой новости...
  Примерный JSON оверхед на элемент: 20 токенов
  Подсчет токенов: обработано 500/1000...
  Подсчет токенов: обработано 1000/1000...
Предварительный подсчет токенов завершен. Заняло: 107.27s
Отправка батча (198 новостей, ~14945 токенов). Запрос #1
DEBUG: response.parts существует. Количество частей: 1
DEBUG: Часть 0: <class 'google.ai.generativelanguage_v1beta.types.content.Part'>
  Текст части 0: '[
  {
    "id": 88230,
    "multi_labels": [
      "Культура"
    ],
    "hier_label": [
      "Культура",
      "Музыка"
    ]
  },
  {
    "id": 60779,
    "multi_labels": [
      "Экономика",
     ...' (всего 36091 символов)
DEBUG: response_text после response.text: '[
  {
    "id": 88230,
    "multi_labels": [
      "Культура"
    ],
    "hier_label": [
      "Культура

In [10]:
df_processed.head(50)

,index,date,year,month,day,text,multi_labels,hier_label
88230,88230,2022-03-28 15:59:56,2022,3,28,Музыкальный фестиваль Park Live перенесен на 2023,[],[]
60779,60779,2021-08-20 17:59:04,2021,8,20,"Путин заявил, что РФ готова после 2024 г осуществлять транзит газа через Украину, объем и сроки контрактов будут зависеть от потребностей ЕС",[],[]
61944,61944,2021-09-02 12:38:08,2021,9,2,"Итальянское Хотя в группе 4 хоббита, волшебник, эльф, гном и 2 людей, руководит ими и несет кольцо - вопреки традиции - не Фродо, а Сэм. Он рассказал РИА Новости, что в обычной жизни зовется Николас Джентиле и работает пекарем-кондитером.\n\nКто-то из участников экспедиции – банковский клерк, кто-то зарабатывает SMM, иной – рабочий, кто-то учится в университете. Кольцо они несут через полстраны.",[],[]
127336,127336,2023-09-05 16:41:44,2023,9,5,"Инициатива о запрете меховой индустрии за один день набрала в Финляндии необходимые 50 тысяч голосов, теперь ее рассмотрит парламент.\n\nВ прошлом месяце в стране выступили с идеей закрепить в конституции права животных, она также поступила в парламент и ждет голосования.",[],[]
71221,71221,2021-11-29 11:47:05,2021,11,29,"13 заражений штаммом ""омикрон"" обнаружили в португальской футбольной команде ""Белененсеш"".\n\nВ субботу ""Белененсеш"" вышел на матч против ""Бенфики"" вдевятером. Во втором тайме (счет к тому моменту был уже 0:7 в пользу ""Бенфики"") на поле появились уже всего 7 игроков.\n\nПосле травмы одного из них матч остановили (вшестером уже нельзя играть по правилам).",[],[]
52319,52319,2021-06-09 09:51:01,2021,6,9,"Комментатор телеканал ""Матч ТВ"" Нобель Арустамян не получил аккредитацию на Евро-2020. Для работы на турнире, который пройдет в 11 городах разных стран, журналисты должны получить одобрение всех оргкомитетов стран-хозяек. Такого одобрения нет от оргкомитета Баку.\n\nВ Ассоциации футбольных федераций Азербайджана этот отказ связали с тем, что журналист «неоднократно совершал незаконные поездки на оккупированные азербайджанские земли» без разрешения властей страны, «проявив неуважение к территориальной целостности».\n\nГенеральный продюсер ""Матч ТВ"" Тина Канделаки назвала ситуацию ""нарушением духа международного спортивного движения"", отметив, что телеканал делает все возможное, чтобы разрешить эту ситуацию.",[],[]
98448,98448,2022-07-16 09:56:04,2022,7,16,"Шойгу проинспектировал российские группировки войск ""Юг"" и ""Центр"", выполняющие задачи специальной военной операции на Украине",[],[]
35758,35758,2021-01-07 00:56:08,2021,1,7,"Пенс поставил на заставку своего личного Twitter фото Байдена с Харрис, но от Трампа не отписался",[],[]
16068,16068,2020-06-03 13:21:13,2020,6,3,"Кремль надеется, что компетентные ведомства примут меры к ""Здесь можно выразить только крайнее сожаление в связи с тем, что такая игра доступна на территории нашей страны. И доступна тем более сейчас, в год, когда мы отмечаем 75-летие Победы. Потому что, конечно, эта игра, с нашей точки зрения, не что иное, как просто пропаганда нацизма. Что недопустимо""",[],[]
62545,62545,2021-09-08 10:58:13,2021,9,8,Горячая линия по взрыву в Ногинске +7 495 542 21 41,[],[]


In [11]:
# from google import genai

# import config  # Импортируем файл конфигурации

# client = genai.Client(api_key=config.API_KEY)  # Используем API ключ из конфигурации

# response = client.models.generate_content(
#     model="gemini-2.0-flash",
#     contents="Здарова бандит",
# )

# print(response.text)